In [2]:
import bisect
from typing import List, Tuple
import itertools
from BoyerMoore import BoyerMoore
from Index_objects import Index, SubseqIndex, queryIndex, querySubseqIndex

### Utilities

In [2]:
def readFasta(filename: str) -> str:
    '''
    Reads fasta files
    Input: fasta filename 
    Output: the sequence as a string
    '''
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            if not line.startswith('>'):
                genome += line.strip()
    return genome

def readFastq(filename: str) -> Tuple[List[str], List[str]]:
    '''
    Reads fastq files
    Input: fastq filename
    Output:
        -- the sequence as a string
        -- the associated qualities of base calls
    '''
    sequences = []
    qualities = []
    with open(filename) as fh:
        for i, line in enumerate(fh):
            if i % 4 == 1:
                sequences.append(line.strip())
            elif i % 4 == 3:
                qualities.append(line.strip())
    return sequences, qualities

def reverseComplement(s: str) -> str:
    '''
    Gives the reverse complement of a pattern
    Input: s, string
    Output: reverse complement of string
    '''
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A', 'N': 'N'}
    t = ''
    for base in s:
        t = complement[base] + t
    return t

def phred33toQ(qual: str) -> int: 
    '''
    Conversion of quality scores to numerical values
    Input: qual, ascii char
    Output: numerical value of quality
    '''
    return ord(qual)-33

### Read alignment (pattern matching) algorithms

#### $\circ$ Naive (brute force) methods

In [3]:
def naive_exact(p: str, t: str) -> List[int]:
    '''
    Naive brute-force method for exact matching of pattern p with text t.
    Compares the p to every offset in the t.
    Input: p and t
    Output: offsets for the occurrences of p within t
    '''
    occurrences = []
    for i in range(len(t) - len(p) + 1):    # loop over alignments
        match = True
        for j, char in enumerate(p):    # loop over characters
            if t[i+j] != char:          # compare characters
                match = False
                break
        if match:
            occurrences.append(i)       # all chars matched; record
    return occurrences


def naive_n_mm(p: str, t: str, n: int) -> List[int]:
    '''
    Naive brute-force method for matching of pattern p with text t allowing for n mismatches.
    Compares the p to every offset in the t.
    Input: p, t, and n
    Output: offsets for the occurrences of p within t allowing n mismatches
    '''
    occurrences = []

    for i in range(len(t) - len(p) + 1):  # loop over alignments
        match = True
        mm=0
        for j in range(len(p)):  # loop over characters
            if t[i+j] != p[j]:  # compare characters
                mm+=1
                if mm>n:
                    match=False
                    break
        if match:
            occurrences.append(i)  # all chars matched; record
    return occurrences

def naive_exact_rc(p: str, t: str) -> List[int]:
    '''
    Naive brute-force method for exact matching of pattern p or its reverse complement with text t.
    Compares the p to every offset in the t.
    Input: p, t
    Output: offsets for the occurrences of p or its reverse complement within t
    '''
    occurrences = naive_exact(p,t)
    p_rc=reverseComplement(p)
    if p!=p_rc:
        occurrences.extend(naive_exact(p_rc,t))
    return occurrences


#### $\circ$ Boyer-Moore algorithm -- exact matching

In [7]:
def boyer_moore_exact_matching(pattern: str, p_bm: object, text: str) -> List[int]:
    """
    Boyer-Moore exact matching of pattern p with text t.
    Use both good suffix and bad character rules to get the best skip distance
    Inputs: pattern=pattern, text=text, p_bm=BoyerMoore object for pattern
    Output: offsets for occurrences of pattern in text
    """
    occurrences = []
    i = 0
    while i < len(text) - len(pattern) + 1:
        shift = 1
        mismatched = False
        for j in range(len(pattern) - 1, -1, -1):
            if pattern[j] != text[i + j]:
                skip_bc = p_bm.bad_character_rule(j, text[i + j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences

#### $\circ$ Approximate matching

In [45]:
def approximate_matching(pattern, text, n, method='SubStringIndex'):
    ''' 
    Approximate matching of pattern with text, allowing n mismatches.
    Uses Boyer-Moore for the exact matching part.
    Returns offsets for occurrence of approximate matches of pattern in text.
    '''
    segment_len = len(pattern) // (n + 1)
    all_matches = set()
    total_hits = 0
    for i in range(n + 1):
        start = i * segment_len
        end = min((i + 1) * segment_len, len(pattern))

        if method == 'BoyerMoore':
            pattern_bm = BoyerMoore(pattern[start:end])
            matches = boyer_moore_exact_matching(pattern[start:end], pattern_bm, text)
        elif method == 'SubStringIndex':
            k = end - start
            text_ind = Index(text, k)
            matches = queryIndex(pattern[start:end], text, text_ind)
        elif method == 'SubSeqIndex':
            k = end - start
            text_ind = SubseqIndex(text, k, 3)
            matches = querySubseqIndex(pattern[start:end], text, text_ind)
        
        else: 
            raise Exception('Not a valid method!')
        
        total_hits += len(matches)
        for m in matches:
            if m < start or m - start + len(pattern) > len(text):
                continue
            mismatches = 0
            for j in range(0, start):
                if pattern[j] != text[m - start + j]:
                    mismatches += 1 
                    if mismatches > n:
                        break
            for j in range(end, len(pattern)):
                if pattern[j] != text[m - start + j]:
                    mismatches += 1 
                    if mismatches > n:
                        break
            if mismatches <= n:
                all_matches.add(m - start)
    print('total hits: ', total_hits)
    return list(all_matches)


#### $\circ$ Edit Distance

In [9]:
def editDist_xy(x, y):
    ''' 
    Find edit distance between patterns x and y
    Returns the edit distance.
    '''
    # Create distance matrix
    D = []
    for i in range(len(x)+1):
        D.append([0]*(len(y)+1))
    # Initialize first row and column of matrix
    for i in range(len(x)+1):
        D[i][0] = i
    for i in range(len(y)+1):
        D[0][i] = i
    # Fill in the rest of the matrix
    for i in range(1, len(x)+1):
        for j in range(1, len(y)+1):
            distHor = D[i][j-1] + 1
            distVer = D[i-1][j] + 1
            if x[i-1] == y[j-1]:
                distDiag = D[i-1][j-1]
            else:
                distDiag = D[i-1][j-1] + 1
            D[i][j] = min(distHor, distVer, distDiag)
    # Edit distance is the value in the bottom right corner of the matrix
    return D[-1][-1]

def editDist_min(p, t):
    ''' 
    Find edit distance between the pattern p and the best match of pattern p within t
    Returns the edit distance.
    '''
    # Create distance matrix: row x column = (p+1) x t
    D = []
    for i in range(len(p)+1):
        D.append([0]*(len(t)+1))
    # Initialize first row and column of matrix
    for i in range(len(p)+1):
        # Distance between empty string and a pattern is equal to the pattern size
        D[i][0] = i
    
    # Set first row to zero: we consider all possible offsets
    # since we want to find the best match of p in t
    for i in range(len(t)+1):
        D[0][i] = 0

    # Fill in the rest of the matrix
    for i in range(1, len(p)+1):
        for j in range(1, len(t)+1):
            distHor = D[i][j-1] + 1
            distVer = D[i-1][j] + 1
            if p[i-1] == t[j-1]:
                distDiag = D[i-1][j-1]
            else:
                distDiag = D[i-1][j-1] + 1
            D[i][j] = min(distHor, distVer, distDiag)

    # Edit distance is the min value in the bottom row of the matrix
    return min(D[-1][:])


### Assembly algorithms

In [108]:
def get_kmers(read, k):
    kmers = []
    for i in range(len(read) - k + 1):  # for each k-mer
        kmers.append(read[i:i+k])
    return kmers

class index(object):
    def __init__(self):
        self.dct={}

    def get_dct(self):
        return self.dct
        
    def add_read(self, read, k):
        kmers = get_kmers(read, k)
        for kmer in kmers:  # for each k-mer
            if kmer in self.dct:
                self.dct[kmer].add(read)
            else:
                self.dct[kmer]=set([read]) 
    

def overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. 
    """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's prefix in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match

def scs(ss):
    """ Returns shortest common superstring of given
        strings, which must be the same length """
    shortest_sup = None
    shortest_sup_lst = []
    for ssperm in itertools.permutations(ss):
        sup = ssperm[0]  # superstring starts as first string
        for i in range(len(ss)-1):
            # overlap adjacent strings A and B in the permutation
            olen = overlap(ssperm[i], ssperm[i+1], min_length=1)
            # add non-overlapping portion of B to superstring
            sup += ssperm[i+1][olen:]
        if shortest_sup is None or len(sup) <= len(shortest_sup):
            shortest_sup = sup  # found shorter superstring
            shortest_sup_lst.append(sup)
    return shortest_sup_lst  # return shortest


def pick_max_overlap(reads, k):
    reada, readb = None, None
    best_olen = 0
    for a, b in itertools.permutations(reads,2):
        olen = overlap(a, b, min_length=k)
        if olen>best_olen:
            reada, readb, best_olen = a, b, olen
    return reada, readb, best_olen

def greedy_scs(reads, k):
    reada, readb, olen = pick_max_overlap(reads, k)
    while olen>0:
        print(reada, readb, reads)
        reads.remove(reada)
        reads.remove(readb)
        reads.append(reada+readb[olen:])
        reada, readb, olen = pick_max_overlap(reads, k)
    return ''.join(reads)

def pick_max_overlap_index(reads, k):
    ind = index()
    for read in reads:
        ind.add_read(read,k)
    kmer_dct = ind.get_dct()   
    reada, readb = None, None
    best_olen = 0
    for read in reads:
        read_suffix = read[-k:]
        for matched_read in kmer_dct[read_suffix]:
            if matched_read != read:
                olen = overlap(read, matched_read, k)
                if olen>best_olen:
                    best_olen = olen 
                    reada, readb = read, matched_read
    return reada, readb, best_olen

def greedy_scs_using_index(reads, k):
    reada, readb, olen = pick_max_overlap_index(reads, k)
    while olen>0:
        # print(reada, readb, reads,olen)
        reads.remove(reada)
        reads.remove(readb)
        reads.append(reada+readb[olen:])
        reada, readb, olen = pick_max_overlap_index(reads, k)
        # print(reada, readb, reads,olen)
    return ''.join(reads)

### Tests

In [11]:
data=readFasta('./chr1.GRCh38.excerpt.fasta')
# print(data)

In [12]:
print(editDist_min('GATTTACCAGATTGAG',data))

2


In [13]:
seq,qual = readFastq('./ERR266411_1.for_asm.fastq')

print(len(seq[0]))

100


In [14]:
# Count read pairs from overlap using a k-mer index

ind = index()

for read in seq:
    ind.add_read(read,30)

print(len(ind.dct.keys()))
kmer_dct = ind.get_dct()

read_pairs=0
num_reads_with_overlap = 0
for read in seq:
    read_suffix = read[-30:]
    i = 0
    # print(len(read_suffix))
    for matched_read in kmer_dct[read_suffix]:
        # print(read,'\n',matched_read,'\n',read_suffix)
        if matched_read != read:
            ov = overlap(read, matched_read, 30)
            # print(ov)
            if ov>=30: 
                read_pairs += 1 #.append((read,matched_read))
                i = 1
    num_reads_with_overlap += i
        

print(read_pairs, num_reads_with_overlap)


108344
904746 7161


In [15]:
overlap('ATGCAAT','CAATCCC',2)

4

In [106]:
# sups=greedy_scs(['CAT', 'CTT', 'TGC', 'TGG', 'GAT', 'ATT'],2)
sups=greedy_scs_using_index(['CAT', 'CTT', 'TGC', 'TGG', 'GAT', 'ATT'],2)
# for val in sups: print(len(val))
print(sups, len(sups))

CAT ATT ['CAT', 'CTT', 'TGC', 'TGG', 'GAT', 'ATT'] 2
None None ['CTT', 'TGC', 'TGG', 'GAT', 'CATT'] 0
CTTTGCTGGGATCATT 16


In [ ]:
sequences,qualities=readFastq('./ERR037900_1.first1000.fastq')

In [ ]:
[phred33toQ(xx) for xx in qualities[0]]


2

In [ ]:
qcvals=[0]*100

for i in range(len(sequences)):
    for j in range(len(sequences[i])):
        qcvals[j] += phred33toQ(qualities[i][j])

In [ ]:
print([x for x in range(len(qcvals)) if qcvals[x]==min(qcvals)])

[66]


In [115]:
seqs, quals = readFastq('./ads1_week4_reads.fq')
print(len(quals[0]))

100


In [116]:
assembled_genome = greedy_scs_using_index(seqs, 5)
print(len(assembled_genome))

15894


In [120]:
As=0
Ts=0
for val in assembled_genome:
    if val=='A': As+=1
    elif val=='T': Ts+=1
print(As, Ts)

print(len(assembled_genome.split('A'))-1, len(assembled_genome.split('T'))-1)



4633 3723
4633 3723


In [ ]:
pattern='GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'

p_bm=BoyerMoore(pattern)

In [ ]:
occ_bm,num_bm=boyer_moore(pattern,p_bm, data)

In [ ]:
bm_matches = approximate_matching_bm('GGCGCGGTGGCTCACGCCTGTAAT', data, 2)
print(len(bm_matches))

19


In [ ]:
ind_matches = approximate_matching_index('GGCGCGGTGGCTCACGCCTGTAAT', data, 2, 4)
print(len(ind_matches))

32


In [ ]:
naive_matches=naive('GGCGCGGTGGCTCACGCCTGTAAT',data)
print(naive_matches[1])

799977


In [ ]:
bm=approximate_matching_bm('GGCGCGGTGGCTCACGCCTGTAAT',data, 2)
print(len(bm))

total hits:  90
19


In [ ]:
indx=approximate_matching_index('GGCGCGGTGGCTCACGCCTGTAAT',data, 2)
print(len(indx))

total hits:  90
19


In [ ]:
subseq_ind = approximate_matching_SubseqIndex('GGCGCGGTGGCTCACGCCTGTAAT',data, 2)

total hits:  0


In [ ]:
print(editDistance('TATTGGCTATACGGTT', 'GCGTATGC'))

11


In [ ]:
a=set([2,4,5,3,4,5])

In [ ]:
a.add(3)
a

{2, 3, 4, 5, 8}

In [48]:
p=approximate_matching('AGTATCGCCT','AGTCATGGATAGGATGGGCCCAAATAC', 7, method='SubStringIndex') #SubSeqIndex
print(len(p))

total hits:  54
8
